# Animierte Mesh-Vergleiche
Dieses Notebook erstellt rotierende GIF-Animationen der verschiedenen 3D-Rekonstruktionsmethoden.

In [13]:
import pandas as pd
import numpy as np
import trimesh
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import os
from pathlib import Path
import sys
import fast_simplification

# GPU/offscreen rendering & IO
import open3d as o3d
import imageio.v2 as imageio
from PIL import Image, ImageDraw, ImageFont
import pyvista as pv

sys.path.append("../")
from TestEvaluationPipeline.mesh_utils import MeshUtils

In [2]:
def find_mesh_file(model_dir, base_name, exts=(".stl", ".obj")):
    """
    Look for a file in model_dir whose name contains base_name and ends with one of the given extensions.
    """
    base = Path(model_dir)
    # try scanning for any file matching pattern *base_name*ext
    for ext in exts:
        pattern = f"*{base_name}*{ext}"
        matches = list(base.glob(pattern))
        if matches:
            return str(matches[0])
    # fallback to any file containing base_name
    globbed = list(base.glob(f"*{base_name}*.*"))
    if globbed:
        return str(globbed[0])
    raise FileNotFoundError(f"No mesh file containing '{base_name}' in '{model_dir}'")


def set_axes_equal_and_zoom(ax, mesh, zoom=1.0):
    xyz = mesh.vertices
    mins = xyz.min(axis=0)
    maxs = xyz.max(axis=0)
    center = (mins + maxs) / 2
    half = (maxs - mins).max() * zoom / 2

    ax.set_xlim(center[0] - half, center[0] + half)
    ax.set_ylim(center[1] - half, center[1] + half)
    ax.set_zlim(center[2] - half, center[2] + half)
    ax.set_box_aspect((1, 1, 1))

In [ ]:
def create_animated_mesh_comparison(
    basenames,
    model_dirs,
    model_names=None,
    exts=(".stl", ".obj"),
    zoom=0.8,
    figsize=(15, 10),
    frames=36,  # Anzahl der Frames für 360° Rotation
    fps=6,  # Frames per second
    output_file="mesh_comparison.gif",
    colormap_name="cividis",  # cooler, colorblind-friendly colormap
):
    """
    Erstellt eine animierte GIF-Datei mit rotierenden Meshes.

    basenames       : list of mesh name substrings WITHOUT extension
    model_dirs      : list of directory paths (first one = GT)
    model_names     : optional list of column titles
    exts            : tuple of extensions to try (default .stl and .obj)
    zoom            : <1 zooms in (meshes look larger), >1 zooms out
    figsize         : figure size for plt.figure
    frames          : number of animation frames (36 = 10° per frame)
    fps             : frames per second for the GIF
    output_file     : filename for the output GIF
    colormap_name   : matplotlib colormap name for depth coloring (cool palette recommended)
    """
    n_rows = len(basenames)
    n_cols = len(model_dirs)

    if model_names is None:
        model_names = [os.path.basename(os.path.normpath(d)) for d in model_dirs]

    # Lade und verarbeite alle Meshes einmal
    meshes = {}
    for i, name in enumerate(basenames):
        # load GT for this row
        gt_path = find_mesh_file(model_dirs[0], name, exts)
        gt = trimesh.load(gt_path)
        meshes[(i, 0)] = gt

        for j, model_dir in enumerate(model_dirs[1:], 1):
            pred_path = find_mesh_file(model_dir, name, exts)
            pred = trimesh.load(pred_path)

            # Simplify if too complex - Special handling for different models
            model_name = os.path.basename(model_dir)

            if "hunyuan" in model_name.lower():
                # Hunyuan3D: preserve much more detail; only simplify very large meshes
                orig_count = pred.vertices.shape[0]
                if orig_count >= 60_000:
                    target_vertices = min(40_000, max(20_000, orig_count // 2))
                    reduction_ratio = 1 - (target_vertices / orig_count)
                    # cap reduction to max 40% for high quality
                    reduction_ratio = max(0.1, min(0.4, reduction_ratio))

                    new_verts, new_faces = fast_simplification.simplify(
                        points=pred.vertices.view(np.ndarray),
                        triangles=pred.faces.view(np.ndarray),
                        target_reduction=reduction_ratio,
                    )
                    pred = trimesh.Trimesh(vertices=new_verts, faces=new_faces)
                    print(
                        f"Hunyuan3D: Simplified from {orig_count} to {pred.vertices.shape[0]} vertices (high-quality)"
                    )
                else:
                    # no simplification for smaller Hunyuan meshes
                    pass
            else:
                # Other models - reduce less aggressively to avoid low-poly look
                orig_count = pred.vertices.shape[0]
                if orig_count >= 10_000:
                    target_vertices = min(8_000, max(6_000, orig_count // 2))
                    reduction_ratio = 1 - (target_vertices / orig_count)
                    reduction_ratio = max(
                        0.2, min(0.6, reduction_ratio)
                    )  # 20-60% reduction

                    new_verts, new_faces = fast_simplification.simplify(
                        points=pred.vertices.view(np.ndarray),
                        triangles=pred.faces.view(np.ndarray),
                        target_reduction=reduction_ratio,
                    )
                    pred = trimesh.Trimesh(vertices=new_verts, faces=new_faces)
                    print(
                        f"{model_name}: Simplified from {orig_count} to {pred.vertices.shape[0]} vertices"
                    )

            # Align to GT
            aligned, _ = MeshUtils.align_icp(pred, gt)
            mesh = trimesh.Trimesh(vertices=aligned.vertices, faces=pred.faces)
            meshes[(i, j)] = mesh

    # Erstelle die Animation
    fig, axes = plt.subplots(
        n_rows, n_cols, subplot_kw={"projection": "3d"}, figsize=figsize, squeeze=False
    )

    # Setup axes styling
    plots = {}
    for i in range(n_rows):
        for j in range(n_cols):
            ax = axes[i, j]
            ax.grid(False)
            ax.xaxis.pane.fill = False
            ax.yaxis.pane.fill = False
            ax.zaxis.pane.fill = False
            for spine in (ax.xaxis, ax.yaxis, ax.zaxis):
                spine.pane.set_edgecolor("none")
            ax.set_axis_off()

            # Title on first row
            if i == 0:
                ax.set_title(model_names[j], fontsize=12, pad=20)

            # Add mesh name on first column (but not for GT column)
            if j == 0 and False:  # Disabled text labels
                ax.text2D(
                    0.02,
                    0.5,
                    basenames[i].replace("_", "\n"),
                    transform=ax.transAxes,
                    rotation=90,
                    va="center",
                    ha="left",
                    fontsize=8,
                )

    def animate(frame):
        # Clear all axes
        for i in range(n_rows):
            for j in range(n_cols):
                axes[i, j].clear()
                ax = axes[i, j]

                # Reapply styling
                ax.grid(False)
                ax.xaxis.pane.fill = False
                ax.yaxis.pane.fill = False
                ax.zaxis.pane.fill = False
                for spine in (ax.xaxis, ax.yaxis, ax.zaxis):
                    spine.pane.set_edgecolor("none")
                ax.set_axis_off()

                # Title on first row
                if i == 0:
                    ax.set_title(model_names[j], fontsize=12, pad=20)

                # Add mesh name on first column (but not for GT column)
                if j == 0 and False:  # Disabled text labels
                    ax.text2D(
                        0.02,
                        0.5,
                        basenames[i].replace("_", "\n"),
                        transform=ax.transAxes,
                        rotation=90,
                        va="center",
                        ha="left",
                        fontsize=8,
                    )

                # Get mesh and plot
                mesh = meshes[(i, j)]

                # Calculate rotation angle
                angle = (frame / frames) * 360

                # Create depth-based coloring with enhanced visual appeal
                # Use Z-coordinate for depth mapping
                z_coords = mesh.vertices[:, 2]
                z_min, z_max = z_coords.min(), z_coords.max()

                # Normalize Z coordinates to 0-1 range for colormap
                if z_max > z_min:
                    z_normalized = (z_coords - z_min) / (z_max - z_min)
                else:
                    z_normalized = np.zeros_like(z_coords)

                # Use specified colormap (default cividis for cool tones)
                colormap = getattr(plt.cm, colormap_name, plt.cm.cividis)

                # Create the surface plot with Z-based coloring
                surf = ax.plot_trisurf(
                    mesh.vertices[:, 0],
                    mesh.vertices[:, 1],
                    mesh.vertices[:, 2],
                    triangles=mesh.faces,
                    cmap=colormap,
                    alpha=0.95,
                    linewidth=0,
                    edgecolor="none",
                )

                # Set the color values for each vertex (normalized 0..1)
                surf.set_array(z_normalized)

                # Set viewing angle
                ax.view_init(elev=20, azim=angle)

                # Set equal aspect & zoom
                set_axes_equal_and_zoom(ax, mesh, zoom=zoom)

        return []

    # Create animation
    print(f"Erstelle Animation mit {frames} Frames...")
    anim = FuncAnimation(fig, animate, frames=frames, interval=1000 // fps, blit=False)

    # Save as GIF mit spezifischer Auflösung
    print(f"Speichere GIF als '{output_file}' mit 960x720 Auflösung...")
    writer = PillowWriter(fps=fps)
    # Set DPI to ensure 960x720 resolution
    fig.set_dpi(75)
    anim.save(output_file, writer=writer, dpi=75)

    plt.close(fig)
    print(
        f"Animation gespeichert! Dateigröße: {os.path.getsize(output_file) / 1024 / 1024:.1f} MB"
    )

    return anim

In [8]:
# GPU-accelerated renderer using Open3D (offscreen)
def create_animated_mesh_comparison_gpu(
    basenames,
    model_dirs,
    model_names=None,
    exts=(".stl", ".obj"),
    zoom=0.8,
    width=960,
    height=720,
    frames=36,
    fps=6,
    output_file="mesh_comparison_gpu.gif",
    colormap_name="cividis",
):
    """
    GPU/offscreen animierter Vergleich mit Open3D. Schneller als Matplotlib und nutzt GPU, falls verfügbar.
    """
    if model_names is None:
        model_names = [os.path.basename(os.path.normpath(d)) for d in model_dirs]

    n_rows = len(basenames)
    n_cols = len(model_dirs)

    # Load and align meshes once (reuse across frames)
    meshes = {}
    for i, name in enumerate(basenames):
        gt_path = find_mesh_file(model_dirs[0], name, exts)
        gt_tm = trimesh.load(gt_path)
        gt_o3d = o3d.geometry.TriangleMesh(
            o3d.utility.Vector3dVector(gt_tm.vertices),
            o3d.utility.Vector3iVector(gt_tm.faces),
        )
        gt_o3d.compute_vertex_normals()
        meshes[(i, 0)] = gt_o3d

        for j, model_dir in enumerate(model_dirs[1:], 1):
            pred_path = find_mesh_file(model_dir, name, exts)
            pred_tm = trimesh.load(pred_path)

            # Align to GT
            aligned, _ = MeshUtils.align_icp(pred_tm, gt_tm)
            pred_tm_aligned = trimesh.Trimesh(
                vertices=aligned.vertices, faces=pred_tm.faces
            )

            # Convert to Open3D
            pred_o3d = o3d.geometry.TriangleMesh(
                o3d.utility.Vector3dVector(pred_tm_aligned.vertices),
                o3d.utility.Vector3iVector(pred_tm_aligned.faces),
            )
            pred_o3d.compute_vertex_normals()
            meshes[(i, j)] = pred_o3d

    # Prepare color map (per-vertex z-based)
    import matplotlib.cm as cm

    colormap = getattr(cm, colormap_name, cm.cividis)

    # Open3D rendering setup
    render = o3d.visualization.rendering
    mat = render.MaterialRecord()
    mat.shader = "defaultLit"  # lit shading
    mat.base_color = (1.0, 1.0, 1.0, 1.0)
    mat.base_roughness = 0.6
    mat.base_metallic = 0.0
    mat.base_reflectance = 0.1

    # Create one renderer and one scene per subplot to avoid re-adding geometry each frame
    scenes = []
    cameras = []
    for i in range(n_rows):
        row_scenes = []
        row_cams = []
        for j in range(n_cols):
            scene = render.Open3DScene(
                render.OffscreenRenderer(width // n_cols, height // n_rows).scene
            )
            row_scenes.append(scene)
            row_cams.append(None)
        scenes.append(row_scenes)
        cameras.append(row_cams)

    # Helper to color mesh by Z in Open3D
    def color_by_z(o3d_mesh):
        verts = np.asarray(o3d_mesh.vertices)
        z = verts[:, 2]
        z_min, z_max = z.min(), z.max()
        if z_max > z_min:
            zn = (z - z_min) / (z_max - z_min)
        else:
            zn = np.zeros_like(z)
        colors = colormap(zn)[:, :3]
        o3d_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
        return o3d_mesh

    # Build scenes once
    for i in range(n_rows):
        for j in range(n_cols):
            mesh = meshes[(i, j)].clone()
            mesh = color_by_z(mesh)
            # Compute a padded bbox for zoom control
            aabb = mesh.get_axis_aligned_bounding_box()
            center = aabb.get_center()
            extent = max(aabb.get_extent()) * zoom
            # Create a scene and add mesh
            r = render.OffscreenRenderer(width // n_cols, height // n_rows)
            sc = r.scene
            sc.set_background([1, 1, 1, 0])  # transparent
            sc.scene.set_lighting(
                render.Open3DScene.LightingProfile.SOFT_SHADOWS, [0, 0, 0]
            )
            sc.add_geometry(f"m_{i}_{j}", mesh, mat)
            # Camera setup
            cam = sc.camera
            sc.setup_camera(60.0, aabb, center)
            cameras[i][j] = (r, sc, cam, center, extent)
            scenes[i][j] = sc

    # Render frames and stitch into a single image per frame
    all_frames = []
    for f in range(frames):
        angle = (f / frames) * 360.0
        row_imgs = []
        for i in range(n_rows):
            col_imgs = []
            for j in range(n_cols):
                r, sc, cam, center, extent = cameras[i][j]
                # Orbit around Z-axis
                theta = np.deg2rad(angle)
                radius = extent * 2.5
                eye = center + np.array(
                    [radius * np.cos(theta), radius * np.sin(theta), extent * 0.6]
                )
                up = np.array([0, 0, 1])
                sc.camera.look_at(center, eye, up)
                img = r.render_to_image()
                img_np = np.asarray(img)
                col_imgs.append(img_np)
            row_imgs.append(np.concatenate(col_imgs, axis=1))
        frame_img = np.concatenate(row_imgs, axis=0)
        all_frames.append(frame_img)

    # Write GIF
    imageio.mimsave(output_file, all_frames, fps=fps)
    print(f"GPU GIF saved to {output_file} ({len(all_frames)} frames)")
    return output_file

In [ ]:
# PyVista fallback if Open3D offscreen is not supported on this platform
def create_animated_mesh_comparison_pv(
    basenames,
    model_dirs,
    model_names=None,
    exts=(".stl", ".obj"),
    zoom=0.8,
    width=960,
    height=720,
    frames=36,
    fps=6,
    output_file="mesh_comparison_pv.gif",
    colormap_name="cividis",
):
    if model_names is None:
        model_names = [os.path.basename(os.path.normpath(d)) for d in model_dirs]

    n_rows = len(basenames)
    n_cols = len(model_dirs)

    # Load and align meshes (trimesh)
    meshes = {}
    for i, name in enumerate(basenames):
        gt_path = find_mesh_file(model_dirs[0], name, exts)
        gt_tm = trimesh.load(gt_path)
        meshes[(i, 0)] = gt_tm
        for j, model_dir in enumerate(model_dirs[1:], 1):
            pred_path = find_mesh_file(model_dir, name, exts)
            pred_tm = trimesh.load(pred_path)
            aligned, _ = MeshUtils.align_icp(pred_tm, gt_tm)
            meshes[(i, j)] = trimesh.Trimesh(
                vertices=aligned.vertices, faces=pred_tm.faces
            )

    # Precompute centers and extents for camera control
    centers_extents = {}
    for i in range(n_rows):
        for j in range(n_cols):
            tm = meshes[(i, j)]
            mins = tm.vertices.min(axis=0)
            maxs = tm.vertices.max(axis=0)
            center = (mins + maxs) / 2.0
            extent = (maxs - mins).max() * zoom
            centers_extents[(i, j)] = (center, extent)

    # Convert trimesh -> PyVista PolyData with z-based scalar
    pv_grids = []
    for i in range(n_rows):
        row = []
        for j in range(n_cols):
            tm = meshes[(i, j)]
            z = tm.vertices[:, 2]
            z_min, z_max = z.min(), z.max()
            zn = (z - z_min) / (z_max - z_min) if z_max > z_min else np.zeros_like(z)
            faces = np.hstack(
                [
                    np.full((tm.faces.shape[0], 1), 3, dtype=np.int64),
                    tm.faces.astype(np.int64),
                ]
            )
            faces = faces.ravel(order="C")
            pv_mesh = pv.PolyData(tm.vertices.astype(np.float64), faces)
            pv_mesh["zcolor"] = zn.astype(np.float64)
            row.append(pv_mesh)
        pv_grids.append(row)

    # Prepare plotter once (offscreen)
    pv.global_theme.background = "white"
    plotter = pv.Plotter(
        shape=(n_rows, n_cols), off_screen=True, window_size=(width, height)
    )

    frames_imgs = []
    for f in range(frames):
        angle_deg = (f / frames) * 360.0
        theta = np.deg2rad(angle_deg)
        plotter.clear()
        for i in range(n_rows):
            for j in range(n_cols):
                plotter.subplot(i, j)
                mesh = pv_grids[i][j]
                plotter.add_mesh(
                    mesh,
                    scalars="zcolor",
                    cmap=colormap_name,
                    smooth_shading=True,
                    show_scalar_bar=False,
                    clim=(0.0, 1.0),
                )
                center, extent = centers_extents[(i, j)]
                radius = max(extent, 1e-6) * 2.5
                eye = center + np.array(
                    [radius * np.cos(theta), radius * np.sin(theta), extent * 0.6]
                )
                # Set camera explicitly
                plotter.camera.position = tuple(eye.tolist())
                plotter.camera.focal_point = tuple(center.tolist())
                plotter.camera.up = (0.0, 0.0, 1.0)
                plotter.enable_parallel_projection()
        img = plotter.screenshot(return_img=True)
        frames_imgs.append(img)

    imageio.mimsave(output_file, frames_imgs, fps=fps)
    print(f"PyVista GIF saved to {output_file} ({len(frames_imgs)} frames)")
    plotter.close()
    return output_file

In [5]:
# Mesh-Namen definieren
filenames = [
    "17781_Common_thyme_Thymus_vulgaris_pollen_grain",
    "17803_Ox-eye_daisy_Leucanthemum_vulgare_pollen_grain",
    "21555_Hard_rush_Juncus_inflexus_pollen_grain_shrunken",
    "21188_Meadow_goats_beard_Tragopogon_pratensis_pollen_grain",
    "17878_Alder_Alnus_sp_pollen_grain_pentaporate",
]

### Ground Truth Spin Animation
Erstellt eine separate GIF-Animation nur der Ground-Truth Meshes (normalisiert & zentriert), analog zu den bestehenden Pixel2Mesh / PixelNeRF / Hunyuan Spins.

In [ ]:
import math
from typing import List
import numpy as np
import trimesh
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import colorsys

try:
    filenames
except NameError:
    filenames = [
        "17781_Common_thyme_Thymus_vulgaris_pollen_grain",
        "17803_Ox-eye_daisy_Leucanthemum_vulgare_pollen_grain",
        "21555_Hard_rush_Juncus_inflexus_pollen_grain_shrunken",
        "21188_Meadow_goats_beard_Tragopogon_pratensis_pollen_grain",
        "17878_Alder_Alnus_sp_pollen_grain_pentaporate",
    ]

try:
    find_mesh_file
except NameError:
    from pathlib import Path
    import os
    def find_mesh_file(model_dir, base_name, exts=(".stl", ".obj")):
        base = Path(model_dir)
        for ext in exts:
            pattern = f"*{base_name}*{ext}"
            matches = list(base.glob(pattern))
            if matches:
                return str(matches[0])
        globbed = list(base.glob(f"*{base_name}*.*"))
        if globbed:
            return str(globbed[0])
        raise FileNotFoundError(f"No mesh file containing '{base_name}' in '{model_dir}'")


def normalize_to_unit_sphere(mesh: trimesh.Trimesh) -> trimesh.Trimesh:
    m = mesh.copy()
    bmin, bmax = m.bounds
    center = (bmin + bmax) / 2.0
    m.vertices -= center
    radii = np.linalg.norm(m.vertices, axis=1)
    max_r = radii.max() if radii.size else 1.0
    if max_r > 0:
        m.vertices /= (2 * max_r)
    c2 = (m.bounds[0] + m.bounds[1]) / 2.0
    m.vertices -= c2
    return m


def hsv_color(i: int, n: int) -> tuple[float, float, float]:
    h = (i / max(1, n)) % 1.0
    s, v = 0.25, 0.95
    return colorsys.hsv_to_rgb(h, s, v)


def create_spin_grid_white(
    basenames: List[str],
    title: str,
    out_file: str,
    gt_dir: str = "../data/processed/meshes",
    exts=(".stl", ".obj"),
    frames: int = 120,
    fps: int = 15,
    elev: float = 25.0,
    bg=(1.0, 1.0, 1.0),
    fast_mode: bool = False,
    downsample_target: int = 15000,
):
    """Allgemeine weiße Spin-Grid Routine.
    fast_mode=True reduziert Frames (z.B. 60) & nutzt niedrigere dpi für schnellere Erstellung.
    downsample_target: Zielanzahl Vertices pro Mesh (vereinfachte Darstellung)"""
    meshes = []
    for name in basenames:
        path = find_mesh_file(gt_dir, name, exts)
        tm = trimesh.load(path)
        # leichte Vereinfachung, wenn groß
        if tm.vertices.shape[0] > downsample_target:
            try:
                tm = tm.simplify_quadratic_decimation(downsample_target)
            except Exception:
                pass
        tm = normalize_to_unit_sphere(tm)
        meshes.append((name, tm))

    if fast_mode:
        frames = min(frames, 60)
        fps = min(fps, 12)

    n = len(meshes)
    cols = math.ceil(math.sqrt(n))
    rows = math.ceil(n / cols)
    cell_extent = 0.6
    scale = 2.4 if fast_mode else 3.0
    fig_w = scale * cols
    fig_h = scale * rows + (0.3 if title else 0)

    fig, axes = plt.subplots(rows, cols, figsize=(fig_w, fig_h), subplot_kw={"projection": "3d"})
    if rows == 1 and cols == 1:
        axes = np.array([[axes]])
    elif rows == 1:
        axes = np.array([axes])
    elif cols == 1:
        axes = axes[:, None]

    fig.patch.set_facecolor(bg)

    def setup_ax(ax):
        ax.set_facecolor(bg)
        ax.set_axis_off()
        ax.grid(False)
        ax.set_xlim(-cell_extent, cell_extent)
        ax.set_ylim(-cell_extent, cell_extent)
        ax.set_zlim(-cell_extent, cell_extent)
        ax.set_box_aspect((1, 1, 1))

    for row in axes:
        for ax in row:
            setup_ax(ax)

    def draw_mesh(ax, mesh: trimesh.Trimesh, color):
        ax.plot_trisurf(
            mesh.vertices[:, 0],
            mesh.vertices[:, 1],
            mesh.vertices[:, 2],
            triangles=mesh.faces,
            color=color,
            linewidth=0.1,
            edgecolor="none",
            alpha=0.97,
        )

    for idx, (name, mesh) in enumerate(meshes):
        r, c = divmod(idx, cols)
        ax = axes[r, c]
        draw_mesh(ax, mesh, hsv_color(idx, n))
        ax.view_init(elev=elev, azim=0)

    if title:
        fig.suptitle(title, color="black", fontsize=14 if fast_mode else 16, weight="bold", y=1.0)
    plt.tight_layout()

    def update(frame):
        angle = 360.0 * frame / frames
        for idx, (name, mesh) in enumerate(meshes):
            r, c = divmod(idx, cols)
            ax = axes[r, c]
            ax.cla()
            setup_ax(ax)
            draw_mesh(ax, mesh, hsv_color(idx, n))
            ax.view_init(elev=elev, azim=angle)
        return []

    anim = FuncAnimation(fig, update, frames=frames, interval=1000 // fps, blit=False)
    writer = PillowWriter(fps=fps)
    dpi = 90 if fast_mode else 110
    anim.save(out_file, writer=writer, dpi=dpi)
    plt.close(fig)
    print(f"Spin Grid gespeichert: {out_file} (frames={frames}, fps={fps}, fast={fast_mode})")
    return out_file

# Schnelleres GT neu rendern
create_spin_grid_white(filenames, title="GROUND TRUTH", out_file="gt_spin_unit_white_fast.gif", fast_mode=True)


### White Background High-Resolution Model Spins
Spins der Rekonstruktionsmodelle im exakt gleichen Stil wie der weiße GT-Spin (Einheitskugel, keine Labels, schwarzer Titel, hohe DPI).

In [ ]:
import os, math, numpy as np, trimesh, colorsys
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

MODEL_RENDER_DIRS = {
    "PIXELNERF": "../TestEvaluationPipeline/data/pollen_augmentation2",
    "HUNYUAN3D": "../TestEvaluationPipeline/data/Hunyuan3D-two-views",
    "PIX2VOX": "../TestEvaluationPipeline/data/pix2vox_aug",
    "PIXEL2MESH++": "../TestEvaluationPipeline/data/refine_p2mpp_augmentation_2_inputs",
}

spin_model_filenames = filenames  # reuse existing selection

try:
    find_mesh_file
except NameError:
    from pathlib import Path
    def find_mesh_file(model_dir, base_name, exts=(".stl", ".obj")):
        base = Path(model_dir)
        for ext in exts:
            pattern = f"*{base_name}*{ext}"
            matches = list(base.glob(pattern))
            if matches:
                return str(matches[0])
        raise FileNotFoundError(f"No mesh for {base_name} in {model_dir}")


def normalize_to_unit_sphere(mesh: trimesh.Trimesh) -> trimesh.Trimesh:
    m = mesh.copy()
    bmin, bmax = m.bounds
    center = (bmin + bmax) / 2.0
    m.vertices -= center
    r = np.linalg.norm(m.vertices, axis=1)
    max_r = r.max() if r.size else 1.0
    if max_r > 0:
        m.vertices /= (2 * max_r)
    c2 = (m.bounds[0] + m.bounds[1]) / 2.0
    m.vertices -= c2
    return m

def hsv_color(i: int, n: int):
    h = (i / max(1, n)) % 1.0
    s, v = 0.25, 0.95
    return colorsys.hsv_to_rgb(h, s, v)


def render_white_spin(model_name: str, model_dir: str, basenames, out_file: str,
                      frames: int = 180, fps: int = 15, elev: float = 25.0,
                      bg=(1,1,1), dpi: int = 140):
    meshes = []
    for name in basenames:
        try:
            p = find_mesh_file(model_dir, name)
            tm = trimesh.load(p)
            tm = normalize_to_unit_sphere(tm)
            meshes.append((name, tm))
        except Exception as e:
            print(f"[WARN] {model_name} fehlgeschlagen {name}: {e}")
    if not meshes:
        print(f"[SKIP] {model_name}: keine Meshes")
        return None

    n = len(meshes)
    cols = math.ceil(math.sqrt(n))
    rows = math.ceil(n / cols)
    cell_extent = 0.6
    fig_w = 3.2 * cols
    fig_h = 3.2 * rows + 0.35
    fig, axes = plt.subplots(rows, cols, figsize=(fig_w, fig_h), subplot_kw={"projection": "3d"})
    if rows == 1 and cols == 1:
        axes = np.array([[axes]])
    elif rows == 1:
        axes = np.array([axes])
    elif cols == 1:
        axes = axes[:, None]

    fig.patch.set_facecolor(bg)

    def setup_ax(ax):
        ax.set_facecolor(bg)
        ax.set_axis_off()
        ax.grid(False)
        ax.set_xlim(-cell_extent, cell_extent)
        ax.set_ylim(-cell_extent, cell_extent)
        ax.set_zlim(-cell_extent, cell_extent)
        ax.set_box_aspect((1,1,1))

    for row_axes in axes:
        for ax in row_axes:
            setup_ax(ax)

    def draw_mesh(ax, mesh, color):
        ax.plot_trisurf(mesh.vertices[:,0], mesh.vertices[:,1], mesh.vertices[:,2],
                        triangles=mesh.faces, color=color, linewidth=0, edgecolor='none', alpha=0.97)

    for idx, (_, mesh) in enumerate(meshes):
        r, c = divmod(idx, cols)
        ax = axes[r, c]
        draw_mesh(ax, mesh, hsv_color(idx, n))
        ax.view_init(elev=elev, azim=0)

    fig.suptitle(model_name, color='black', fontsize=16, weight='bold', y=1.0)
    plt.tight_layout()

    def update(frame):
        angle = 360.0 * frame / frames
        for idx, (_, mesh) in enumerate(meshes):
            r, c = divmod(idx, cols)
            ax = axes[r, c]
            ax.cla(); setup_ax(ax)
            draw_mesh(ax, mesh, hsv_color(idx, n))
            ax.view_init(elev=elev, azim=angle)
        return []

    anim = FuncAnimation(fig, update, frames=frames, interval=1000//fps, blit=False)
    writer = PillowWriter(fps=fps)
    anim.save(out_file, writer=writer, dpi=dpi)
    plt.close(fig)
    print(f"Saved: {out_file}")
    return out_file

white_spin_results = {}
for model_name, model_dir in MODEL_RENDER_DIRS.items():
    out = f"{model_name.lower().replace('+','plus')}_spin_unit_white.gif"
    white_spin_results[model_name] = render_white_spin(model_name, model_dir, spin_model_filenames, out)

white_spin_results

In [2]:
# Weiße Spin-Grids für weitere Modelle (fast)
model_dirs_map = {
    "PIX2VOX": "../TestEvaluationPipeline/data/pix2vox_aug",
    "PIXEL2MESH++": "../TestEvaluationPipeline/data/refine_p2mpp_augmentation_2_inputs",
    "HUNYUAN3D": "../TestEvaluationPipeline/data/Hunyuan3D-two-views",
    "PIXELNERF": "../TestEvaluationPipeline/data/pollen_augmentation2",  # falls erneut gewünscht
}

white_spin_outputs = {}
for title, mdir in model_dirs_map.items():
    try:
        out_name = f"{title.lower().replace('+','p').replace('3d','').replace(' ','_')}_spin_unit_white_fast.gif"
        res = create_spin_grid_white(
            basenames=filenames,
            title=title,
            out_file=out_name,
            gt_dir=mdir,
            fast_mode=True,
            downsample_target=12000,
        )
        white_spin_outputs[title] = res
    except Exception as e:
        print(f"[ERROR] {title}: {e}")

white_spin_outputs

[ERROR] PIX2VOX: name 'create_spin_grid_white' is not defined
[ERROR] PIXEL2MESH++: name 'create_spin_grid_white' is not defined
[ERROR] HUNYUAN3D: name 'create_spin_grid_white' is not defined
[ERROR] PIXELNERF: name 'create_spin_grid_white' is not defined


{}

## Methodenvergleich - Alle 5 besten Modelle
Vergleicht Ground Truth, Visual Hull, Pix2Vox, Hunyuan3D, Pixel2Mesh++ und PixelNeRF

In [ ]:
model_dirs_all = [
    "../data/processed/meshes",
    "../TestEvaluationPipeline/data/vh_2img",
    "../TestEvaluationPipeline/data/pix2vox_aug",
    "../TestEvaluationPipeline/data/Hunyuan3D-two-views",
    "../TestEvaluationPipeline/data/refine_p2mpp_augmentation_2_inputs",
    "../TestEvaluationPipeline/data/pollen_augmentation2",
]

model_names_all = [
    "GT",
    "Visual Hull",
    "Pix2Vox",
    "Hunyuan3D",
    "Pixel2Mesh++",
    "PixelNeRF",
]

# Erstelle große Übersichts-Animation (nur 3 Meshes für bessere Performance)
filenames_subset = [
    "17781_Common_thyme_Thymus_vulgaris_pollen_grain",
    "17803_Ox-eye_daisy_Leucanthemum_vulgare_pollen_grain",
    "21188_Meadow_goats_beard_Tragopogon_pratensis_pollen_grain",
]

# MEMORY TEST: Nur 1 Mesh für minimalen Speicherverbrauch
filenames_test = [
    "17781_Common_thyme_Thymus_vulgaris_pollen_grain",
]

print("🧪 MEMORY TEST (GPU → PyVista-Fallback falls nötig): 1 Mesh...")
try:
    create_animated_mesh_comparison_gpu(
        filenames_test,
        model_dirs_all,
        model_names_all,
        zoom=0.65,
        width=960,
        height=240,  # kompakter, schneller
        frames=12,  # mehr als 6 aber immer noch schnell
        fps=6,
        output_file="test_single_mesh_gpu.gif",
        colormap_name="cividis",
    )
except Exception as e:
    print(f"Open3D offscreen nicht verfügbar: {e}\n→ Fallback auf PyVista")
    create_animated_mesh_comparison_pv(
        filenames_test,
        model_dirs_all,
        model_names_all,
        zoom=0.65,
        width=960,
        height=240,
        frames=12,
        fps=6,
        output_file="test_single_mesh_pv.gif",
        colormap_name="cividis",
    )

print("\n🎬 FULL COMPARISON (GPU → PyVista-Fallback falls nötig): 3 Meshes...")
try:
    create_animated_mesh_comparison_gpu(
        filenames_subset,
        model_dirs_all,
        model_names_all,
        zoom=0.65,
        width=960,
        height=720,
        frames=18,  # 20° Schritte
        fps=9,
        output_file="all_methods_comparison_gpu.gif",
        colormap_name="cividis",
    )
except Exception as e:
    print(f"Open3D offscreen nicht verfügbar: {e}\n→ Fallback auf PyVista")
    create_animated_mesh_comparison_pv(
        filenames_subset,
        model_dirs_all,
        model_names_all,
        zoom=0.65,
        width=960,
        height=720,
        frames=18,
        fps=9,
        output_file="all_methods_comparison_pv.gif",
        colormap_name="cividis",
    )

# Optional: CPU-Version weiter verfügbar (langsamer)
print("\n🧪 MEMORY TEST (CPU): 1 Mesh...")
create_animated_mesh_comparison(
    filenames_test,
    model_dirs_all,
    model_names_all,
    zoom=0.65,
    figsize=(10.0, 2.5),
    frames=6,
    fps=3,
    output_file="test_single_mesh_hires.gif",
    colormap_name="cividis",
)

🧪 MEMORY TEST (GPU → PyVista-Fallback falls nötig): 1 Mesh...
Open3D offscreen nicht verfügbar: [Open3D Error] (__cdecl open3d::visualization::rendering::EngineInstance::EngineInstance(void)) D:\a\Open3D\Open3D\cpp\open3d\visualization\rendering\filament\FilamentEngine.cpp:104: EGL Headless is not supported on this platform.

→ Fallback auf PyVista
Open3D offscreen nicht verfügbar: [Open3D Error] (__cdecl open3d::visualization::rendering::EngineInstance::EngineInstance(void)) D:\a\Open3D\Open3D\cpp\open3d\visualization\rendering\filament\FilamentEngine.cpp:104: EGL Headless is not supported on this platform.

→ Fallback auf PyVista


In [ ]:
# Run VRAM-friendly VTK streaming animation (loads per tile/frame)
print("🎥 Streaming-Renderer (VTK): Lädt pro Tile/Frame nur ein Mesh → VRAM-schonend")
create_grid_video_vtk_streaming(
    basenames=filenames_subset,
    model_dirs=model_dirs_all,
    model_names=model_names_all,
    width=960,
    height=540,
    frames=18,
    fps=9,
    output_file="grid_stream_gpu.mp4",
    bg=(1, 1, 1),
)

## Schnelle Single-Mesh Animation

In [ ]:
# Erstelle eine schnelle Animation für nur ein Mesh
single_mesh = ["17781_Common_thyme_Thymus_vulgaris_pollen_grain"]

create_animated_mesh_comparison(
    single_mesh,
    model_dirs_all,
    model_names_all,
    zoom=0.6,
    figsize=(12.8, 3.6),  # 960x270 für single row bei 75 DPI
    frames=48,  # Mehr Frames für smoothere Animation
    fps=12,  # Höhere FPS
    output_file="single_mesh_comparison.gif",
    colormap_name="cividis",  # cool, colorblind-friendly palette
)

Hunyuan3D: Simplified from 35 to 35 vertices (conservative)
pollen_augmentation2: Simplified from 5090 to 5090 vertices
Erstelle Animation mit 48 Frames...
Speichere GIF als 'single_mesh_comparison.gif' mit 960x720 Auflösung...


In [ ]:
# Zeige die erstellten GIF-Dateien
import glob

gif_files = glob.glob("*.gif")
print("Erstellte GIF-Dateien:")
for gif in gif_files:
    size_mb = os.path.getsize(gif) / 1024 / 1024
    print(f"  {gif} - {size_mb:.1f} MB")

Erstellte GIF-Dateien:
  all_methods_comparison.gif - 4.2 MB
  pix2vox_comparison.gif - 10.2 MB
  pixelnerf_comparison.gif - 1.6 MB
  single_mesh_comparison.gif - 2.1 MB


In [1]:
# High-resolution static visualization for Hunyuan3D
def create_static_hires_comparison(mesh_name, model_dirs, model_names, figsize=(20, 8)):
    """Create static high-resolution comparison showing full detail"""

    fig, axes = plt.subplots(
        2, len(model_dirs), figsize=figsize, subplot_kw={"projection": "3d"}
    )

    for j, (model_dir, model_name) in enumerate(zip(model_dirs, model_names)):
        # High-res view (top row)
        ax_hires = axes[0, j]
        # Simplified view (bottom row)
        ax_simple = axes[1, j]

        try:
            mesh_path = find_mesh_file(model_dir, mesh_name)
            if mesh_path:
                mesh = trimesh.load(mesh_path)

                # Original high-res mesh (top)
                ax_hires.plot_trisurf(
                    mesh.vertices[:, 0],
                    mesh.vertices[:, 1],
                    mesh.vertices[:, 2],
                    triangles=mesh.faces,
                    alpha=0.85,
                    cmap="cividis",
                )
                ax_hires.set_title(
                    f"{model_name}\nFull Resolution\n({mesh.vertices.shape[0]:,} vertices)",
                    fontsize=10,
                )

                # Simplified mesh (bottom) - show what animation uses
                if "hunyuan" in model_dir.lower() and mesh.vertices.shape[0] >= 60_000:
                    target_vertices = min(
                        40_000, max(20_000, mesh.vertices.shape[0] // 2)
                    )
                    reduction_ratio = 1 - (target_vertices / mesh.vertices.shape[0])
                    reduction_ratio = max(0.1, min(0.4, reduction_ratio))
                elif mesh.vertices.shape[0] >= 10_000:
                    target_vertices = min(
                        8_000, max(6_000, mesh.vertices.shape[0] // 2)
                    )
                    reduction_ratio = 1 - (target_vertices / mesh.vertices.shape[0])
                    reduction_ratio = max(0.2, min(0.6, reduction_ratio))
                else:
                    reduction_ratio = 0

                if reduction_ratio > 0:
                    new_verts, new_faces = fast_simplification.simplify(
                        points=mesh.vertices.view(np.ndarray),
                        triangles=mesh.faces.view(np.ndarray),
                        target_reduction=reduction_ratio,
                    )
                    simple_mesh = trimesh.Trimesh(vertices=new_verts, faces=new_faces)

                    ax_simple.plot_trisurf(
                        simple_mesh.vertices[:, 0],
                        simple_mesh.vertices[:, 1],
                        simple_mesh.vertices[:, 2],
                        triangles=simple_mesh.faces,
                        alpha=0.85,
                        cmap="cividis",
                    )
                    ax_simple.set_title(
                        f"Animation Version\n({simple_mesh.vertices.shape[0]:,} vertices)",
                        fontsize=10,
                    )
                else:
                    ax_simple.plot_trisurf(
                        mesh.vertices[:, 0],
                        mesh.vertices[:, 1],
                        mesh.vertices[:, 2],
                        triangles=mesh.faces,
                        alpha=0.85,
                        cmap="cividis",
                    )
                    ax_simple.set_title(
                        f"Animation Version\n(No simplification)", fontsize=10
                    )

                # Styling
                for ax in [ax_hires, ax_simple]:
                    ax.view_init(elev=20, azim=45)
                    ax.set_xlabel("X", fontsize=8)
                    ax.set_ylabel("Y", fontsize=8)
                    ax.set_zlabel("Z", fontsize=8)

        except Exception as e:
            for ax in [ax_hires, ax_simple]:
                ax.text(
                    0.5,
                    0.5,
                    0.5,
                    f"Error: {str(e)[:20]}...",
                    transform=ax.transAxes,
                    ha="center",
                    va="center",
                )

    plt.suptitle(
        f"High-Resolution vs Animation Comparison: {mesh_name}",
        fontsize=14,
        fontweight="bold",
    )
    plt.tight_layout()
    return fig


print("🔍 HIGH-RESOLUTION COMPARISON:")
print("Top row: Full resolution meshes")
print("Bottom row: Simplified versions used in animations")
print("=" * 60)

# Show high-res comparison for one mesh
test_mesh = "17781_Common_thyme_Thymus_vulgaris_pollen_grain"
hires_fig = create_static_hires_comparison(
    test_mesh, model_dirs_all, model_names_all, figsize=(24, 10)
)
plt.show()

🔍 HIGH-RESOLUTION COMPARISON:
Top row: Full resolution meshes
Bottom row: Simplified versions used in animations


NameError: name 'model_dirs_all' is not defined

In [ ]:
import argparse, os, math, glob, sys, subprocess
import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy
import imageio_ffmpeg as ioff


def read_polydata(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".stl":
        r = vtk.vtkSTLReader()
    elif ext == ".obj":
        r = vtk.vtkOBJReader()
    else:
        raise ValueError(f"Unsupported extension: {ext}")
    r.SetFileName(path)
    r.Update()
    # Compute normals for decent shading
    n = vtk.vtkPolyDataNormals()
    n.SetInputConnection(r.GetOutputPort())
    n.SplittingOff()
    n.ConsistencyOn()
    n.AutoOrientNormalsOn()
    n.Update()
    return n.GetOutput()


def make_actor(poly, target_diag=1.0, color=(0.8, 0.8, 0.85)):
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(poly)
    mapper.ScalarVisibilityOff()
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)

    # Normalize scale to target diagonal
    b = poly.GetBounds()  # xmin,xmax,ymin,ymax,zmin,zmax
    dx, dy, dz = b[1] - b[0], b[3] - b[2], b[5] - b[4]
    diag = max(1e-9, math.sqrt(dx * dx + dy * dy + dz * dz))
    scale = target_diag / diag
    actor.SetScale(scale, scale, scale)

    # Rotate around its own center
    cx, cy, cz = (0.5 * (b[0] + b[1]), 0.5 * (b[2] + b[3]), 0.5 * (b[4] + b[5]))
    actor.SetOrigin(cx, cy, cz)

    # Simple material
    actor.GetProperty().SetColor(*color)
    actor.GetProperty().SetInterpolationToGouraud()
    actor.GetProperty().SetSpecular(0.1)
    actor.GetProperty().SetSpecularPower(10)
    return actor


def color_palette(i, n):
    # pleasant HSV ramp -> RGB
    h = (i / max(1, n)) % 1.0
    s, v = 0.25, 0.95
    import colorsys

    return colorsys.hsv_to_rgb(h, s, v)


def main():
    ap = argparse.ArgumentParser()
    ap.add_argument(
        "--in",
        dest="inputs",
        nargs="+",
        required=True,
        help="file paths or globs (e.g., meshes/*.stl)",
    )
    ap.add_argument("--out", default="spin_grid.mp4")
    ap.add_argument(
        "--cols", type=int, default=None, help="grid columns (default=sqrt(n))"
    )
    ap.add_argument("--fps", type=int, default=30)
    ap.add_argument(
        "--frames", type=int, default=360, help="frames for one full 360° turn"
    )
    ap.add_argument("--width", type=int, default=1920)
    ap.add_argument("--height", type=int, default=1080)
    ap.add_argument("--bg", type=float, nargs=3, default=[0.06, 0.07, 0.08])
    ap.add_argument(
        "--padding", type=float, default=1.4, help="cell spacing multiplier"
    )
    args = ap.parse_args()

    # Expand globs
    files = []
    for pat in args.inputs:
        files += glob.glob(pat)
    files = sorted(
        [
            f
            for f in files
            if os.path.isfile(f) and os.path.splitext(f)[1].lower() in (".stl", ".obj")
        ]
    )
    if not files:
        print("No meshes found.", file=sys.stderr)
        sys.exit(1)

    n = len(files)
    cols = args.cols or math.ceil(math.sqrt(n))
    rows = math.ceil(n / cols)

    # VTK renderer/window (off-screen)
    renderer = vtk.vtkRenderer()
    renderer.SetBackground(*args.bg)

    renwin = vtk.vtkRenderWindow()
    renwin.OffScreenRenderingOn()
    renwin.SetSize(args.width, args.height)
    renwin.AddRenderer(renderer)
    renwin.SetMultiSamples(0)  # faster
    renwin.SetAlphaBitPlanes(0)

    # Light kit for nice basic lighting
    kit = vtk.vtkLightKit()
    kit.AddLightsToRenderer(renderer)
    kit.SetKeyLightIntensity(0.9)
    kit.SetFillLightIntensity(0.6)
    kit.SetBackLightIntensity(0.3)

    # Create actors and lay them out
    actors = []
    target_diag = 1.0
    spacing = args.padding * target_diag
    x0 = -(cols - 1) * spacing * 0.5
    y0 = +(rows - 1) * spacing * 0.5

    for i, path in enumerate(files):
        poly = read_polydata(path)
        c = color_palette(i, n)
        actor = make_actor(poly, target_diag=target_diag, color=c)
        r, cidx = divmod(i, cols)
        x = x0 + cidx * spacing
        y = y0 - r * spacing
        actor.SetPosition(x, y, 0.0)
        actors.append(actor)
        renderer.AddActor(actor)

    # Camera to fit all
    renderer.ResetCamera()
    cam = renderer.GetActiveCamera()
    cam.Roll(0)
    cam.SetViewUp(0, 1, 0)
    # Move a bit back for padding after ResetCamera
    cam.Dolly(1.15)
    renderer.ResetCameraClippingRange()

    # Window-to-image (RGBA)
    w2i = vtk.vtkWindowToImageFilter()
    w2i.SetInput(renwin)
    w2i.ReadFrontBufferOff()
    w2i.SetInputBufferTypeToRGBA()

    # Prepare FFmpeg
    ffmpeg = ioff.get_ffmpeg_exe()
    enc = "h264_nvenc"
    # Probe if NVENC exists; otherwise fall back to libx264
    try:
        out = subprocess.run(
            [ffmpeg, "-hide_banner", "-encoders"], capture_output=True, text=True
        )
        if "h264_nvenc" not in out.stdout:
            enc = "libx264"
    except Exception:
        enc = "libx264"

    cmd = [
        ffmpeg,
        "-y",
        "-f",
        "rawvideo",
        "-vcodec",
        "rawvideo",
        "-pix_fmt",
        "rgba",
        "-s",
        f"{args.width}x{args.height}",
        "-r",
        str(args.fps),
        "-i",
        "-",
        "-an",
        "-vcodec",
        enc,
        "-preset",
        "p5" if enc == "h264_nvenc" else "veryfast",
        "-pix_fmt",
        "yuv420p",
        args.out,
    ]
    proc = subprocess.Popen(cmd, stdin=subprocess.PIPE)

    try:
        for f in range(args.frames):
            angle = 360.0 * f / args.frames
            for a in actors:
                a.SetOrientation(0.0, angle, 0.0)
            renwin.Render()
            w2i.Modified()
            w2i.Update()
            img = w2i.GetOutput()
            w, h = renwin.GetSize()
            arr = vtk_to_numpy(img.GetPointData().GetScalars()).reshape(h, w, 4)
            # Flip vertically because VTK's origin is bottom-left
            arr = np.flipud(arr)
            proc.stdin.write(arr.tobytes())
        proc.stdin.close()
        proc.wait()
    finally:
        if proc.poll() is None:
            proc.kill()

    print(f"Done → {args.out}")


if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] --in INPUTS [INPUTS ...] [--out OUT]
                             [--cols COLS] [--fps FPS] [--frames FRAMES]
                             [--width WIDTH] [--height HEIGHT] [--bg BG BG BG]
                             [--padding PADDING]
ipykernel_launcher.py: error: ambiguous option: --f=c:\Users\super\AppData\Roaming\jupyter\runtime\kernel-v319a9fbc51ea82dfb8a273152c332254764ecfd7c.json could match --fps, --frames


SystemExit: 2

c:\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# VRAM-friendly VTK grid animation: load each mesh individually per tile/frame
import math, subprocess, tempfile, shutil
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy
import matplotlib.cm as cm


def trimesh_to_vtk_polydata(vertices: np.ndarray, faces: np.ndarray, add_zcolor=True):
    # vertices: (N,3), faces: (M,3)
    pts = vtk.vtkPoints()
    pts.SetData(numpy_to_vtk(vertices.astype(np.float64)))
    # Build faces (vtkCellArray)
    cells = vtk.vtkCellArray()
    # VTK 9 accepts numpy arrays via SetCells with offsets/types, but we use InsertNextCell loop for safety
    for tri in faces.astype(np.int64):
        cells.InsertNextCell(3)
        cells.InsertCellPoint(int(tri[0]))
        cells.InsertCellPoint(int(tri[1]))
        cells.InsertCellPoint(int(tri[2]))
    poly = vtk.vtkPolyData()
    poly.SetPoints(pts)
    poly.SetPolys(cells)
    if add_zcolor:
        z = vertices[:, 2]
        zmin, zmax = float(z.min()), float(z.max())
        zn = (z - zmin) / (zmax - zmin) if zmax > zmin else np.zeros_like(z)
        arr = numpy_to_vtk(zn.astype(np.float32))
        arr.SetName("zcolor")
        poly.GetPointData().SetScalars(arr)
    # Normals for shading
    normals = vtk.vtkPolyDataNormals()
    normals.SetInputData(poly)
    normals.SplittingOff()
    normals.ConsistencyOn()
    normals.AutoOrientNormalsOn()
    normals.Update()
    return normals.GetOutput()


def build_cividis_lut(n=256):
    cmap = cm.get_cmap("cividis", n)
    lut = vtk.vtkLookupTable()
    lut.SetNumberOfTableValues(n)
    lut.SetRange(0.0, 1.0)
    for i in range(n):
        r, g, b, a = cmap(i)
        lut.SetTableValue(i, float(r), float(g), float(b), float(a))
    lut.Build()
    return lut


def render_tile_image(
    vertices: np.ndarray,
    faces: np.ndarray,
    angle_deg: float,
    tile_w: int,
    tile_h: int,
    bg=(1.0, 1.0, 1.0),
    target_diag=1.0,
):
    poly = trimesh_to_vtk_polydata(vertices, faces, add_zcolor=True)
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(poly)
    mapper.SetScalarVisibility(True)
    mapper.SetScalarRange(0.0, 1.0)
    mapper.SetLookupTable(build_cividis_lut(256))
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    # Normalize scale to target diag
    b = poly.GetBounds()  # xmin,xmax,ymin,ymax,zmin,zmax
    dx, dy, dz = b[1] - b[0], b[3] - b[2], b[5] - b[4]
    diag = max(1e-9, math.sqrt(dx * dx + dy * dy + dz * dz))
    s = target_diag / diag
    actor.SetScale(s, s, s)
    # Center origin to rotate around center
    cx, cy, cz = (0.5 * (b[0] + b[1]), 0.5 * (b[2] + b[3]), 0.5 * (b[4] + b[5]))
    actor.SetOrigin(cx, cy, cz)
    actor.SetOrientation(0.0, angle_deg, 0.0)  # yaw
    # Renderer/window
    ren = vtk.vtkRenderer()
    ren.SetBackground(*bg)
    ren.AddActor(actor)
    win = vtk.vtkRenderWindow()
    win.OffScreenRenderingOn()
    win.SetMultiSamples(0)
    win.SetSize(int(tile_w), int(tile_h))
    win.AddRenderer(ren)
    ren.ResetCamera()
    cam = ren.GetActiveCamera()
    cam.Dolly(1.2)  # slight zoom-out
    ren.ResetCameraClippingRange()
    win.Render()
    w2i = vtk.vtkWindowToImageFilter()
    w2i.SetInput(win)
    w2i.ReadFrontBufferOff()
    w2i.SetInputBufferTypeToRGBA()
    w2i.Update()
    img = w2i.GetOutput()
    arr = vtk_to_numpy(img.GetPointData().GetScalars()).reshape(tile_h, tile_w, 4)
    arr = np.flipud(arr)
    return arr[:, :, :3]  # RGB


def create_grid_video_vtk_streaming(
    basenames,
    model_dirs,
    model_names=None,
    exts=(".stl", ".obj"),
    zoom=0.8,
    width=960,
    height=540,
    frames=18,
    fps=9,
    output_file="grid_stream_gpu.mp4",
    bg=(1.0, 1.0, 1.0),
):
    """
    Create a grid animation by loading a single mesh per tile/frame (VRAM-friendly).
    Uses trimesh+ICP for alignment, VTK for GPU offscreen rendering, and FFmpeg for encoding.
    """
    if model_names is None:
        model_names = [os.path.basename(os.path.normpath(d)) for d in model_dirs]
    n_rows = len(basenames)
    n_cols = len(model_dirs)
    tile_w = width // n_cols
    tile_h = height // n_rows

    # Prepare FFmpeg (NVENC if available)
    import imageio_ffmpeg as ioff

    ffmpeg = ioff.get_ffmpeg_exe()
    enc = "h264_nvenc"
    try:
        out = subprocess.run(
            [ffmpeg, "-hide_banner", "-encoders"], capture_output=True, text=True
        )
        if "h264_nvenc" not in out.stdout:
            enc = "libx264"
    except Exception:
        enc = "libx264"
    cmd = [
        ffmpeg,
        "-y",
        "-f",
        "rawvideo",
        "-vcodec",
        "rawvideo",
        "-pix_fmt",
        "rgb24",
        "-s",
        f"{width}x{height}",
        "-r",
        str(fps),
        "-i",
        "-",
        "-an",
        "-vcodec",
        enc,
        "-preset",
        "p5" if enc == "h264_nvenc" else "veryfast",
        "-pix_fmt",
        "yuv420p",
        output_file,
    ]
    proc = subprocess.Popen(cmd, stdin=subprocess.PIPE)

    try:
        for f in range(frames):
            angle = 360.0 * f / frames
            row_imgs = []
            for i, name in enumerate(basenames):
                col_imgs = []
                # Load GT once per row to align predictions
                gt_path = find_mesh_file(model_dirs[0], name, exts)
                gt_tm = trimesh.load(gt_path)
                for j, model_dir in enumerate(model_dirs):
                    if j == 0:
                        # Render GT
                        tm = gt_tm
                    else:
                        pred_path = find_mesh_file(model_dir, name, exts)
                        pred_tm = trimesh.load(pred_path)
                        # Align to GT (per tile, avoids keeping all in VRAM)
                        aligned, _ = MeshUtils.align_icp(pred_tm, gt_tm)
                        tm = trimesh.Trimesh(
                            vertices=aligned.vertices, faces=pred_tm.faces
                        )
                    # Render tile image and append
                    rgb = render_tile_image(
                        tm.vertices,
                        tm.faces,
                        angle,
                        tile_w,
                        tile_h,
                        bg=bg,
                        target_diag=1.0,
                    )
                    col_imgs.append(rgb)
                row_imgs.append(np.concatenate(col_imgs, axis=1))
            frame_img = np.concatenate(row_imgs, axis=0)
            proc.stdin.write(frame_img.tobytes())
        proc.stdin.close()
        proc.wait()
    finally:
        if proc.poll() is None:
            proc.kill()
    print(f"Done → {output_file}")
    return output_file

: 